In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from pymysql import Connection
import time

import threading
import concurrent.futures
import re

In [2]:
url = 'https://www.biqg.cc/wangyou/'
r = requests.get(url)
html = r.text

In [3]:
# 删除首页和排行榜，将数据标准化
soup = BeautifulSoup(html, 'html.parser')
li_tags = soup.find_all('li')
li_tags.pop(0) 
li_tags.pop(-1)

# 获取每个标签的href
href_list = []
for li_tag in li_tags:
    a_tag = li_tag.find('a')
    if a_tag:
        href_value = a_tag.get('href')
        href_list.append(href_value)
print(href_list)

['/xuanhuan/', '/wuxia/', '/dushi/', '/lishi/', '/wangyou/', '/kehuan/', '/mm/', '/finish/']


In [4]:
# 拼接出每个标签的url
category_list = []
for href in href_list:
    category_href = urljoin(url,href)
    category_list.append(category_href)
print(category_list)

['https://www.biqg.cc/xuanhuan/', 'https://www.biqg.cc/wuxia/', 'https://www.biqg.cc/dushi/', 'https://www.biqg.cc/lishi/', 'https://www.biqg.cc/wangyou/', 'https://www.biqg.cc/kehuan/', 'https://www.biqg.cc/mm/', 'https://www.biqg.cc/finish/']


In [5]:
# 获取到每本书的url，存在list里
def Get_book_url_list(category_list):
    book_list = []
    for url in category_list:
        # 找到每本书的div
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        book = soup.find_all('div', class_='image')
        # 测试前三本书
        book_3 = book[:3]
        # 获取到每本书的url
        for div in book_3:
            a_tag = div.find('a')
            href_value = a_tag.get('href')
            book_url = urljoin(url, href_value)
            book_list.append(book_url)
    return book_list

book_list = Get_book_url_list(category_list)
print(book_list[-1])
print(book_list)

https://www.biqg.cc/book/147836/
['https://www.biqg.cc/book/140131/', 'https://www.biqg.cc/book/150180/', 'https://www.biqg.cc/book/133656/', 'https://www.biqg.cc/book/131771/', 'https://www.biqg.cc/book/132655/', 'https://www.biqg.cc/book/3212/', 'https://www.biqg.cc/book/135937/', 'https://www.biqg.cc/book/145262/', 'https://www.biqg.cc/book/148773/', 'https://www.biqg.cc/book/26678/', 'https://www.biqg.cc/book/1765/', 'https://www.biqg.cc/book/13768/', 'https://www.biqg.cc/book/162825/', 'https://www.biqg.cc/book/146749/', 'https://www.biqg.cc/book/160443/', 'https://www.biqg.cc/book/1586/', 'https://www.biqg.cc/book/154414/', 'https://www.biqg.cc/book/147390/', 'https://www.biqg.cc/book/94777/', 'https://www.biqg.cc/book/151692/', 'https://www.biqg.cc/book/126052/', 'https://www.biqg.cc/book/154645/', 'https://www.biqg.cc/book/146033/', 'https://www.biqg.cc/book/147836/']


In [8]:
chapter_list = []
for book_url in book_list:
    try:
        # 获取所有章节的dd标签
        r = requests.get(book_url)
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        dd_tag = soup.find_all('dd')
        dd_tag.pop(0)

        for dd in dd_tag:
            # 获取每一章的url,存入chapter_list
            a_tag = dd.find('a')
            href = a_tag.get('href')
            chapter_url = urljoin(url, href)
            if chapter_url.startswith("https://www.biqg.cc/book/"):
                chapter_list.append(chapter_url)
    except Exception as e :
        print("异常："+str(e) )
print(chapter_list)

异常：pop from empty list
异常：pop from empty list
['https://www.biqg.cc/book/140131/1.html', 'https://www.biqg.cc/book/140131/2.html', 'https://www.biqg.cc/book/140131/3.html', 'https://www.biqg.cc/book/140131/4.html', 'https://www.biqg.cc/book/140131/5.html', 'https://www.biqg.cc/book/140131/6.html', 'https://www.biqg.cc/book/140131/7.html', 'https://www.biqg.cc/book/140131/8.html', 'https://www.biqg.cc/book/140131/9.html', 'https://www.biqg.cc/book/140131/10.html', 'https://www.biqg.cc/book/140131/11.html', 'https://www.biqg.cc/book/140131/12.html', 'https://www.biqg.cc/book/140131/13.html', 'https://www.biqg.cc/book/140131/14.html', 'https://www.biqg.cc/book/140131/15.html', 'https://www.biqg.cc/book/140131/16.html', 'https://www.biqg.cc/book/140131/17.html', 'https://www.biqg.cc/book/140131/18.html', 'https://www.biqg.cc/book/140131/19.html', 'https://www.biqg.cc/book/140131/20.html', 'https://www.biqg.cc/book/140131/21.html', 'https://www.biqg.cc/book/140131/22.html', 'https://www.biq

In [ ]:
# 写入txt文件，方便使用
with open('book_url.txt', 'w', encoding='utf8') as f:
    for url in chapter_list:
        f.write(f"{url},")

In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from pymysql import Connection
import time
import threading
import concurrent.futures
import re


with open('book_url.txt', 'r', encoding='utf8') as f:
    # 读取文件内容
    content = f.read()

# 将文本内容按逗号划分成列表
url_list = content.split(',')
print(url_list[965])
print(len(url_list)+1)
print((len(url_list)+1)/67)



In [ ]:
# 创建数据库连接
con = Connection(
    host="localhost",# 主机名
    port=3306, # 端口
    user="root", # 账户
    passwd="hk123" # 密码
)

# 设置游标
cursor = con.cursor()
# 选择数据库
cursor.execute("use library")
# 提交结果
con.commit()

# 创建一个锁
lock = threading.Lock()
# 共享资源
shared_resource = 0
# 因为我爬的笔趣阁有bug，所以我先定义一个有bug的书本列表
bad_list = []

def get_one_book(chapter):
    status = True
    while(status):
        try:
            # 解析页面的html    
            r = requests.get(chapter)
            html = r.text
            soup = BeautifulSoup(html, 'html.parser')

            # 使用正则表达式提取数字
            num_match = re.search(r'/(\d+)/(\d+)\.html$', chapter)
            last_number = num_match.group(2)

            # 找到书名
            name_div = soup.find('div', class_ = 'path wap_none')
            if name_div:
                a = name_div.find_all('a')[1]
            elif a not in bad_list:
                bad_list.append([a.text,chapter])
                print(f"{last_number}name_div是空的！")
                break
            else:break

            

            # 找到标签和内容
            h1_tag = soup.find('h1')
            content_div = soup.find('div', class_ = 'Readarea ReadAjax_content')

            # 找到最后一个<p>标签并删除
            if content_div:
                last_p_tag = content_div.find_all('p')[-1]
                last_p_tag.decompose()
            elif a not in bad_list:
                bad_list.append([a.text,chapter])
                print(f"{last_number}content_div是空的！")
                break
            else:break
            
            # # 写入数据
            print(f"{a.text}, {str(last_number)}, {h1_tag.text}     URL:{chapter}")
            insert_sql = "INSERT INTO book_content (book_name, chapter_num, title, content) VALUES (%s, %s, %s, %s)"
            lock.acquire()
            cursor.execute(insert_sql, (a.text, last_number, h1_tag.text, content_div))
            lock.release()

            status = False
        except Exception as e:
            print(f"异常：{str(e)}----{last_number}")
            time.sleep(5)
            
max_threads = 200  # 最大同时运行线程数
num_start = 0   # 起始下标
num_end = 540   # 结束下标
wic_num = 18    # 起始批次

# 因为程序老是发癫，设置一个起始点
num_start = num_start + 540 * wic_num
num_end = num_end + 540 * wic_num

# 分批爬取并存入数据库
for i in range(54):
    print(str(wic_num) + "znm")
    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        executor.map(get_one_book, url_list[num_start:num_end])
    num_start+=540
    num_end+=540
    con.commit() # 提交结果
    wic_num+=1
    print("====================================================================================l")
    print("=====================================================================================")

con.close() # 关闭连接


In [ ]:
# 以下测试用的，不用管
print(bad_list[0][1].text)

In [ ]:
num = 114101
for i in range(1000):
    if num % (i+1) == 0:
        print(f'{(i+1)}     {num/(i+1)}')